In [2]:
import os

import pandas as pd
from pandas import DataFrame, Series

In [ ]:


CARPETA = r'C:\Users\cesar\OneDrive - Universidad de Burgos\TFG\Repo\CIC_IOT-20250124T124300Z-001\CIC_IOT'

# Listar los archivos de la carpeta
df = DataFrame(os.listdir(CARPETA), columns=['filenames'])

def obtener_prefijo(titulo:str)->str:
    tiene_num=titulo.rsplit('_',1)[-1].removesuffix('.csv').isnumeric()
    if(tiene_num):
        return titulo.rsplit('_',1)[0]
    return titulo.rsplit('.',1)[0] # Quitar solo la extension si no tiene numero

df['prefix'] = df['filenames'].apply(obtener_prefijo)
# Agrupar los archivos por tipo de ataque
df_grouped: DataFrame = df.groupby(
    'prefix')['filenames'].apply(list).reset_index()

df_grouped

,prefix,filenames
0,flows_aggregated_BenignTraffic,"[flows_aggregated_BenignTraffic_0.csv, flows_a..."
1,flows_aggregated_DDoS-HTTP_Flood,[flows_aggregated_DDoS-HTTP_Flood.csv]
2,flows_aggregated_DDoS-SYN,"[flows_aggregated_DDoS-SYN_0.csv, flows_aggreg..."
3,flows_aggregated_DoS-TCP_Flood,"[flows_aggregated_DoS-TCP_Flood_0.csv, flows_a..."
4,flows_aggregated_Recon-OSScan,[flows_aggregated_Recon-OSScan.csv]
5,flows_aggregated_Recon_PortScan,[flows_aggregated_Recon_PortScan.csv]


In [4]:
def filter_filenames(df:DataFrame, text:str)->list[str]:
    return df[df['prefix'].str.contains(text)]['filenames'].iloc[0]


filtered_filenames: list[str] = filter_filenames(df_grouped, '_DoS')
print("Archivos utilizados: ",filtered_filenames)

tipo = '_5'
tiempo_flujo = 30  # Flujos de 30 Segundos
dfs: list[DataFrame] = [pd.read_csv(os.path.join(CARPETA, file),
                                    sep=';', header=0, index_col=False,
                                    usecols=lambda x: x.endswith(tipo) and
                                    (x.startswith('num_flows') or x.startswith('packets_forward_mean') or
                                     x.startswith('avg_pkt_size_mean') or x.startswith('packets_forward_median') or
                                     x.startswith('packets_forward_q1') or x.startswith('packets_forward_q3')))
                        for file in filtered_filenames]

df_combined: DataFrame = pd.concat(dfs, ignore_index=True)

flows_sec:Series = (df_combined['num_flows'+tipo]/tiempo_flujo)

pack_sec: float = (df_combined['packets_forward_mean'+tipo]*flows_sec).mean()

tam_paquetes_sec: float = (df_combined['avg_pkt_size_mean'+tipo]).mean()

pack_sec_q1: float = (df_combined['packets_forward_q1'+tipo]*flows_sec).mean()
pack_sec_q3: float = (df_combined['packets_forward_q3'+tipo]*flows_sec).mean()
pack_sec_median: float = (df_combined['packets_forward_median'+tipo]*flows_sec).mean()

print(f'Se envia una media de {pack_sec:.1f} paquetes por segundo con un tamaño medio de {tam_paquetes_sec:.1f} bytes')
print(f"Donde el 25% de los flujos envian {pack_sec_q1:.1f} paquetes por segundo y el 75% envian {pack_sec_q3:.1f} paquetes por segundo, mediana: {pack_sec_median:.1f}")

Archivos utilizados:  ['flows_aggregated_DoS-TCP_Flood_0.csv', 'flows_aggregated_DoS-TCP_Flood_1.csv', 'flows_aggregated_DoS-TCP_Flood_2.csv', 'flows_aggregated_DoS-TCP_Flood_3.csv', 'flows_aggregated_DoS-TCP_Flood_4.csv', 'flows_aggregated_DoS-TCP_Flood_5.csv', 'flows_aggregated_DoS-TCP_Flood_6.csv', 'flows_aggregated_DoS-TCP_Flood_7.csv']
Se envia una media de 10253.3 paquetes por segundo con un tamaño medio de 127.6 bytes
Donde el 25% de los flujos envian 10134.4 paquetes por segundo y el 75% envian 10389.6 paquetes por segundo, mediana: 10267.4


Benign: 59, 205, 5, 30, 12

DDoS: 418, 180, 5, 77, 19